In [3]:
import pennylane as qml
import numpy as np

# 3 qubits
n_qubits = 3

# Hamiltonian parameters
g = 9.8
w0 = 0.1
w1 = 0.2
t = 1.0  # total evolution time

# Define your Hamiltonian using Pauli terms
coeffs = [
    -g * w0 / 2,  # for X0 X1
    -g * w0 / 2,  # for Y0 Y1
    -g * w1 / 2,  # for X1 X2
    -g * w1 / 2   # for Y1 Y2
]
obs = [
    qml.PauliX(0) @ qml.PauliX(1),
    qml.PauliY(0) @ qml.PauliY(1),
    qml.PauliX(1) @ qml.PauliX(2),
    qml.PauliY(1) @ qml.PauliY(2),
]
H = qml.Hamiltonian(coeffs, obs)

# Initial state |001> (index 1 in 8-dimensional vector)
initial_state = np.zeros(2**n_qubits)
initial_state[1] = 1.0

# PennyLane device
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def ideal_state():
    # Prepare the exact initial state
    qml.StatePrep(initial_state, wires=range(n_qubits))
    # Evolve with e^{-i H t}
    qml.evolve(H)
    # Return the statevector
    return qml.probs()

psi_ideal = ideal_state()
print("Ideal final statevector:", psi_ideal)

Ideal final statevector: [0.         0.0703238  0.52950207 0.         0.40017413 0.
 0.         0.        ]


#### **GUP-modified Optomechanics**

In [9]:
# 3 qubits
n_qubits = 6

# Hamiltonian parameters
g = 9.8
w0 = 0.1
w1 = 0.2
u0 = 0.3
u1 = 0.4
t = 1.0  # total evolution time

# Define your Hamiltonian using Pauli terms
coeffs = [
    -(g * w0 * u0) / 4,
    -(g * w0 * u0) / 4,
    -(g * w0 * u0) / 4,
    -(g * w0 * u0) / 4,

    -(g * w0 * u1) / 4,
    -(g * w0 * u1) / 4,
    -(g * w0 * u1) / 4,
    -(g * w0 * u1) / 4,

    -(g * w1 * u0) / 4,
    -(g * w1 * u0) / 4,
    -(g * w1 * u0) / 4,
    -(g * w1 * u0) / 4,

    -(g * w1 * u1) / 4,
    -(g * w1 * u1) / 4,
    -(g * w1 * u1) / 4,
    -(g * w1 * u1) / 4,   
]
obs = [
    qml.PauliX(0) @ qml.PauliX(2) @ qml.PauliX(1) @ qml.PauliX(3),
    qml.PauliX(0) @ qml.PauliX(2) @ qml.PauliY(1) @ qml.PauliY(3),
    qml.PauliY(0) @ qml.PauliY(2) @ qml.PauliX(1) @ qml.PauliX(3),
    qml.PauliY(0) @ qml.PauliY(2) @ qml.PauliY(1) @ qml.PauliY(3),

    qml.PauliX(0) @ qml.PauliX(2) @ qml.PauliX(3) @ qml.PauliX(5),
    qml.PauliX(0) @ qml.PauliX(2) @ qml.PauliY(3) @ qml.PauliY(5),
    qml.PauliY(0) @ qml.PauliY(2) @ qml.PauliX(3) @ qml.PauliX(5),
    qml.PauliY(0) @ qml.PauliY(2) @ qml.PauliY(3) @ qml.PauliY(5),

    qml.PauliX(2) @ qml.PauliX(4) @ qml.PauliX(1) @ qml.PauliX(3),
    qml.PauliX(2) @ qml.PauliX(4) @ qml.PauliY(1) @ qml.PauliY(3),
    qml.PauliY(2) @ qml.PauliY(4) @ qml.PauliX(1) @ qml.PauliX(3),
    qml.PauliY(2) @ qml.PauliY(4) @ qml.PauliY(1) @ qml.PauliY(3),

    qml.PauliX(2) @ qml.PauliX(4) @ qml.PauliX(3) @ qml.PauliX(5),
    qml.PauliX(2) @ qml.PauliX(4) @ qml.PauliY(3) @ qml.PauliY(5),
    qml.PauliY(2) @ qml.PauliY(4) @ qml.PauliX(3) @ qml.PauliX(5),
    qml.PauliY(2) @ qml.PauliY(4) @ qml.PauliY(3) @ qml.PauliY(5),
]
H = qml.Hamiltonian(coeffs, obs)

# CHANGE FROM HERE

# Initial state |001001>  --> binary 001001 = decimal 9
initial_state = np.zeros(2**n_qubits, dtype=complex)
initial_state[9] = 1.0

# PennyLane device
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def ideal_state():
    qml.StatePrep(initial_state, wires=range(n_qubits))
    qml.evolve(H)
    return qml.state()

psi_ideal = ideal_state()

# Print basis states with amplitudes and probabilities
print("Final state in computational basis:\n")
print("State\t\tAmplitude\t\t\tProbability")
print("-" * 70)

for idx, amp in enumerate(psi_ideal):
    bitstring = format(idx, f"0{n_qubits}b")
    prob = np.abs(amp)**2
    print(f"|{bitstring}> \t{amp:.6f}\t {prob:.6e}")


Final state in computational basis:

State		Amplitude			Probability
----------------------------------------------------------------------
|000000> 	0.000000+0.000000j	 0.000000e+00
|000001> 	0.000000+0.000000j	 0.000000e+00
|000010> 	0.000000+0.000000j	 0.000000e+00
|000011> 	0.000000+0.000000j	 0.000000e+00
|000100> 	0.000000+0.000000j	 0.000000e+00
|000101> 	0.000000+0.000000j	 0.000000e+00
|000110> 	0.000000+0.636286j	 4.048596e-01
|000111> 	0.000000+0.000000j	 0.000000e+00
|001000> 	0.000000+0.000000j	 0.000000e+00
|001001> 	0.652767+0.000000j	 4.261043e-01
|001010> 	0.000000+0.000000j	 0.000000e+00
|001011> 	0.000000+0.000000j	 0.000000e+00
|001100> 	0.000000+0.000000j	 0.000000e+00
|001101> 	0.000000+0.000000j	 0.000000e+00
|001110> 	0.000000+0.000000j	 0.000000e+00
|001111> 	0.000000+0.000000j	 0.000000e+00
|010000> 	0.000000+0.000000j	 0.000000e+00
|010001> 	0.000000+0.000000j	 0.000000e+00
|010010> 	0.000000+0.000000j	 0.000000e+00
|010011> 	0.000000+0.000000j	 0.000000e+00
|